In [1]:
import pandas as pd
from langgraph.graph import StateGraph
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.language_models import BaseChatModel
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import List, TypedDict
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment
import re
import pdfplumber

# 定義狀態字典類型，用於保存每個客戶的狀態信息
class CustomerState(TypedDict):
    news_content: str  # 輸入的新聞內容
    customer_features: List[str]  # 客戶特徵列表
    extracted_features: str  # 提取的客戶特徵摘要
    news_summary: str  # 基於客戶特徵的新聞摘要
    investment_advice_result: str  # 生成的投資建議
    impact_score: int  # 新增的影響程度分數

# 定義個性化新聞代理類
class PersonalizedNewsAgent:

    def __init__(self, model: BaseChatModel, checkpointer: SqliteSaver):
        self.model = model  # 初始化生成式語言模型
        self.checkpointer = checkpointer  # 初始化檢查點保存器，用於保存狀態
        builder = StateGraph(CustomerState)  # 創建StateGraph並傳入狀態字典類型
        builder.add_node("initialize", self.initialize_node)
        builder.add_node("extract_customer_features", self.extract_customer_features_node)
        builder.add_node("personalized_news_summary", self.personalized_news_summary_node)
        builder.add_node("investment_advice", self.investment_advice_node)
        builder.add_node("impact_assessment", self.impact_assessment_node)  # 新增影響評估節點
        builder.add_node("output_results", self.output_results_node)

        builder.set_entry_point("initialize")
        builder.add_edge("initialize", "extract_customer_features")
        builder.add_edge("extract_customer_features", "personalized_news_summary")
        builder.add_edge("personalized_news_summary", "investment_advice")
        builder.add_edge("investment_advice", "impact_assessment")  # 添加新的邊
        builder.add_edge("impact_assessment", "output_results")
        self.graph = builder.compile(checkpointer=checkpointer)

    def initialize_node(self, state: CustomerState):
        return {
            "extracted_features": "",
            "news_summary": "",
            "investment_advice_result": "",
            "impact_score": 50,  # 初始化影響程度分數為 50（中等）
        }

    def extract_customer_features_node(self, state: CustomerState):
        customer_features_text = "\n".join(state["customer_features"])
        messages = [
            SystemMessage(content="以下所有產品皆為基金類型的產品，請直接列點總結每個客戶特徵並確保格式一致，100字內，列點包括投資配置、風險偏好、交易狀況、持倉狀況等，並確保生成內容完全來自以下客戶特徵信息，不參考其他來源。請用繁體中文回答。"),
            HumanMessage(content=customer_features_text),
        ]
        response = self.model.invoke(messages)
        state["extracted_features"] = response.content.strip()
        return {"extracted_features": state["extracted_features"]}

    def personalized_news_summary_node(self, state: CustomerState):
        messages = [
            SystemMessage(content="請根據以下投研報告內容及客戶特徵生成投研報告重點摘要，個人化條列式生成3至5點，與客戶投資產品相關的內容放在最前面，每點字數不超過30字，總字數少於200字，不要包括任何客戶特徵信息，只能從以下新聞內容生成摘要。請用繁體中文回答"),
            HumanMessage(content=f"客戶特徵: {state['extracted_features']}"),
            HumanMessage(content=state["news_content"])
        ]
        response = self.model.invoke(messages)
        state["news_summary"] = response.content.strip()
        return {"news_summary": state["news_summary"]}

    def investment_advice_node(self, state: CustomerState):
        messages = [
            SystemMessage(content="請用繁體中文回答。你是一位具有 CFA Level 3 資格的資深投資專家，請確保投研報告中的資訊有確實反映在投資建議中，投資建議應該考慮到客戶的風險承受能力、投資目標、資產配置偏好以及投研報告內容，務必確保每個建議都是實際可行的，並且根據客戶所持有資產被影響的程度大小排序你列出的每一項建議，不需要說明影響性大小。"),
            AIMessage(content=f"客戶特徵: {state['extracted_features']}"),
            AIMessage(content=f"新聞摘要: {state['news_summary']}"),
        ]
        response = self.model.invoke(messages)
        state["investment_advice_result"] = response.content.strip()
        return {"investment_advice_result": state["investment_advice_result"]}

    def impact_assessment_node(self, state: CustomerState):
        messages = [
        SystemMessage(content="你是一位具有 CFA Level 3 資格的資深投資專家，請評估新聞對每個客戶的影響程度。給出0到100分的評分，分數越高代表影響越大。"),
        AIMessage(content=f"客戶特徵: {state['extracted_features']}"),
        AIMessage(content=f"新聞摘要: {state['news_summary']}"),
    ]
        response = self.model.invoke(messages)
        impact_text = response.content.strip()

        # 初始化默認值
        score = 20
        explanation = "影響甚小，先給予20分"

        # 使用正則表達式提取数字評分
        score_match = re.search(r'\b(\d{1,3})\b', impact_text)
        if score_match:
            score = int(score_match.group(1))
            score = min(max(score, 0), 100)
            explanation = impact_text[impact_text.find(score_match.group(1)) + len(score_match.group(1)):].strip()
            explanation = explanation.replace('*', '').strip()  # 移除可能的米字號或其他不必要符号

        state["impact_score"] = f"{score} 分"
        return {"impact_score": state["impact_score"]}

    def output_results_node(self, state: CustomerState):
        return {
            "extracted_features": state["extracted_features"],
            "news_summary": state["news_summary"],
            "investment_advice_result": state["investment_advice_result"],
            "impact_score": state["impact_score"],  # 包含影響程度分數
        }

# 定義格式化輸出結果的函數
def format_output_for_customer(customer_name, extracted_features, news_summary, investment_advice_result, impact_score):
    return (f"客戶名稱: {customer_name}\n"
            f"提取的客戶特徵:\n{extracted_features}\n\n"
            f"新聞摘要:\n{news_summary}\n\n"
            f"投資建議:\n{investment_advice_result}\n\n"
            f"新聞對客戶的影響程度: {impact_score} 分\n")

# 定義PDF提取摘要的函數
def extract_pdf_summary(pdf_file_path):
    summary = []
    with pdfplumber.open(pdf_file_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            summary.append(text[:200] + "...")  # 提取前200個字作為摘要
    return "\n".join(summary)

# 初始化模型和檢查點保存器
model = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)
memory = SqliteSaver.from_conn_string(":memory:")

bot = PersonalizedNewsAgent(model=model, checkpointer=memory)
    
thread = {"configurable": {"thread_id": "1"}}

# 讀取Excel表格
df = pd.read_excel("teste.xlsx")

# 從PDF提取新聞內容
pdf_summary = extract_pdf_summary("test.pdf")

# 使用 LLM 生成 PDF 的摘要
messages = [
    SystemMessage(content="請將以下PDF的內容生成條列摘要，請用繁體中文回答"),
    HumanMessage(content=pdf_summary)
]

# 調用模型生成PDF摘要
response = model.invoke(messages)
pdf_summary_list = response.content.strip()  # 生成的PDF摘要

# 打印生成的摘要列表，供參考
print("生成的PDF摘要:")
print(pdf_summary_list)

# 定義輸入的新聞內容，將生成的PDF摘要作為新聞內容
state = {
    "news_content": pdf_summary_list
}

# 生成客戶特徵
all_customers_output = []
all_customers_data = []

# 自動讀取Excel欄位名稱，排除ID欄位
selected_columns = df.columns[1:]

# 迭代每個客戶，生成特徵摘要和投資建議
for index, row in df.iterrows():
    customer_name = row['ID']
    
    # 根據選擇的欄位來生成客戶特徵
    customer_features = []
    for col in selected_columns:
        if pd.notna(row[col]):  # 檢查該欄位是否有數值
            feature = f"{col}: {row[col]}"
            customer_features.append(feature)
    
    # 更新狀態
    state["customer_features"] = customer_features

    # 執行圖形流程
    result = bot.graph.invoke(state, thread)

    # 檢查生成的投資建議是否為空，如果是空的，給予警告並記錄
    if not result["investment_advice_result"]:
        print(f"警告: {customer_name} 的投資建議生成為空！")

    # 格式化輸出結果
    customer_output = format_output_for_customer(
        customer_name, 
        result["extracted_features"], 
        result["news_summary"],
        result["investment_advice_result"], 
        result["impact_score"]  # 添加影響程度分數
    )
    
    all_customers_output.append(customer_output)

    all_customers_data.append({
        "客戶": customer_name,
        "客戶特徵": result["extracted_features"],
        "新聞摘要": result["news_summary"],
        "投資建議": result["investment_advice_result"],
        "影響程度分數": result["impact_score"]  # 新增的影響程度分數列
    })

# 創建DataFrame並輸出到Excel
output_df = pd.DataFrame(all_customers_data)
with pd.ExcelWriter("Personalized_Investment_Advice_Output_2.xlsx", engine="openpyxl") as writer:
    output_df.to_excel(writer, sheet_name="Sheet1", index=False)
    worksheet = writer.sheets["Sheet1"]
    
    # 調整欄位寬度
    for i, col in enumerate(output_df.columns):
        max_width = max(output_df[col].astype(str).map(len).max(), len(col)) + 2
        worksheet.column_dimensions[get_column_letter(i + 1)].width = max_width
    
    # 文字換行
    for row in worksheet.iter_rows():
        for cell in row:
            cell.alignment = Alignment(wrap_text=True)

print("Excel 文件已生成並保存，請檢查結果。")


c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\senpai-m2n-YTg9-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


生成的PDF摘要:
## 投資策略週報 (2024/09/02 – 2024/09/06)  條列摘要

**一、 投資策略**

* **債券市場:**
    * 增持金融債，增加存續期間：預期聯準會降息後殖利率將下行，金融債等景氣敏感產業將受惠於利差收斂。
    * 短期美債殖利率再下行空間有限：十年期殖利率與政策利率差值已擴大至歷史低位。
    * 預防性降息有利循環性產業利差收斂：降息將對循環性產業的基本面動能起到托底作用，預期投等債、非投等債利差持續收斂。
* **股票市場:**
    * 股市仍有震盪風險，但可開始留意投資風格轉換。
    * 科技股短期將延續修正行情：Nvidia 財報不如預期，資金更為偏好價值股。
    * 中國股市：儘管滬深300持續走弱，但金融股逆勢上漲，反應銀行業改善的利潤表現，或暗示大盤走勢離落底不遠。
    * 機會市場：
        * 美國：9月總統大選辯論等不確定因素猶存，市場仍有潛在波動風險。若股市再次下跌，可重新將焦點轉向成長股。
        * 中國：儘管近期滬深300持續走弱，但金融類股卻逆勢上漲，反應銀行業改善的利潤表現。過去當金融股止穩，或暗示大盤走勢離落底不遠。
        * 印度：MSCI在新興市場指數當中持續上調印度權重，預期將帶動資金流入。
* **匯市展望:**
    * 日圓：BoJ總裁鷹派發言，帶動日圓重新轉強。
    * 美元：市場過度樂觀聯準會降息空間，導致美元指數近期疲態，短期有望走出反彈格局。

**二、 市場訊息**

* **頭條新聞:**
    * 油價劇烈波動：利比亞將暫停原油生產，但預期長期影響有限。
    * AI 伺服器展望樂觀。
* **產業訊息:**
    * NVIDIA 營收優於預期，但股價並不買單：數據中心業務仍是主要拉動成長的關鍵，但遊戲和 AI PC 業務營收年增率放緩。
* **總體經濟:**
    * 不必擔憂景氣轉向黃紅燈：7月台灣景氣轉向黃紅燈主要受颱風影響，台灣出口動能並未減弱。
* **重點提醒:**
    * 相較於 CPI 數據，市場目前更關注消費、就業市場的變化，本週將揭露服務業 ISM、非農就業、失業率等關鍵數據，將影響 9/18 FOMC 會議對於後續降息路徑的態度。

**三、 外匯&商品市場